Import necessary packages

In [2]:
import osmnx as ox
import networkx as nx
import pandas as pd #To deal with dataframes
import tqdm #Not vital, only necessary to create the loading bar during distance calculation

#Note: the following notebook has been created in Python 3.12.6

General example of network distance between 2 points

In [ ]:
import osmnx as ox
import networkx as nx

# Coordinates of points (latitude, longitude)
point_a = (60.1699, 24.9384)  # Helsinki Central Station
point_b = (60.1719, 24.9415)  # Helsinki Cathedral


# Get a walking network for Helsinki
G = ox.graph_from_point(point_a, dist=2000, network_type='walk')

# Find the nearest nodes to the points
node_a = ox.distance.nearest_nodes(G, X=point_a[1], Y=point_a[0])
node_b = ox.distance.nearest_nodes(G, X=point_b[1], Y=point_b[0])

# Calculate the shortest walking path distance
distance_meters = nx.shortest_path_length(G, node_a, node_b, weight='length')
print(f"Walking distance: {distance_meters} meters")


list_a = [point_a]
list_b = [point_b]
# Define graph and convert point lists to nodes
G = ox.graph_from_point((60.1699, 24.9384), dist=1000, network_type='walk')
nodes_a = [ox.distance.nearest_nodes(G, x[1], x[0]) for x in list_a]
nodes_b = [ox.distance.nearest_nodes(G, x[1], x[0]) for x in list_b]

# Calculate nearest network distance
results = []
for node_a in nodes_a:
    distances = [nx.shortest_path_length(G, node_a, node_b, weight='length') for node_b in nodes_b]
    results.append(min(distances))

print("Closest network distances:", results)


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\osmnx\graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


Walking distance: 311.94 meters


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\osmnx\graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


Closest network distances: [311.94]


### Shortest path length function

The following function develops on the preceding, and is used to calculate the shortest distance between the points contained in two dataframes. Such an information can be useful when conducting different spatial analyses (e.g. as a variable in an hedonic regression)

In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd

# Define the function to calculate the network-based distance
def calculate_network_distance(df1, df2, network_type='walk', dist=2000):
    # Initialize an empty list to store the results
    distances = []
    
    # Get the walking network around the first point in df1 (this can be expanded for larger datasets)
    G = ox.graph_from_point((df1['latitude'].iloc[0], df1['longitude'].iloc[0]), dist=dist, network_type=network_type)
    
    # For each point in df1, find the nearest network node
    df1['node'] = df1.apply(lambda row: ox.distance.nearest_nodes(G, X=row['longitude'], Y=row['latitude']), axis=1)
    
    # For each point in df2, find the nearest network node
    df2['node'] = df2.apply(lambda row: ox.distance.nearest_nodes(G, X=row['longitude'], Y=row['latitude']), axis=1)
    
    # Calculate the shortest path distance between each point in df1 and the closest point in df2
    for _, row1 in tqdm.tqdm(df1.iterrows(), total=len(df1), desc="Calculating shortest distances"):

        # Get the node for the current point in df1
        node_a = row1['node']
        
        # Find the nearest point in df2 and calculate the shortest path distance
        distances_to_b = []
        for _, row2 in df2.iterrows():
            node_b = row2['node']
            try:
                # Calculate the shortest path length (in meters) between the nodes
                distance = nx.shortest_path_length(G, node_a, node_b, weight='length')
                distances_to_b.append(distance)
            except nx.NetworkXNoPath:
                # If no path exists, we can append a high distance value or NaN
                distances_to_b.append(float('inf'))
        
        # Get the minimum distance to any point in df2 for the current point in df1
        distances.append(min(distances_to_b))
    
    # Add the calculated distances to the dataframe df1
    df1['closest_distance'] = distances
    
    return df1
'''
#Example DataFrames with latitudes and longitudes
df1 = pd.DataFrame({
    'latitude': [60.1699, 60.1719],
    'longitude': [24.9384, 24.9425]
})

df2 = pd.DataFrame({
    'latitude': [60.1739, 60.1679],
    'longitude': [24.9415, 24.9365]
})

# Run the function
result_df = calculate_network_distance(df1, df2)

# Print the result
print(result_df)'''


"\n#Example DataFrames with latitudes and longitudes\ndf1 = pd.DataFrame({\n    'latitude': [60.1699, 60.1719],\n    'longitude': [24.9384, 24.9425]\n})\n\ndf2 = pd.DataFrame({\n    'latitude': [60.1739, 60.1679],\n    'longitude': [24.9415, 24.9365]\n})\n\n# Run the function\nresult_df = calculate_network_distance(df1, df2)\n\n# Print the result\nprint(result_df)"

An example using publicly available data on some of Milan's most famous buildings and Milan's metro stations

In [3]:
milan_points = pd.read_csv('points_example.csv')
milan_metro_stations = pd.read_csv('metro_stations.csv')

In [ ]:
print('Milan Points:', '\n', milan_points.head(), '\n\n', 'Milan metro stations:', '\n', milan_metro_stations.head())

Milan Points: 
    id                          point  latitude  longitude
0   1                Duomo di Milano   45.4641     9.1919
1   2                  Sforza Castle   45.4706     9.1796
2   3  Galleria Vittorio Emanuele II   45.4642     9.1900
3   4              Teatro alla Scala   45.4672     9.1896
4   5       Santa Maria delle Grazie   45.4658     9.1702 

 Milan metro stations 
    ID   latitude  longitude
0   1  45.546128   9.437564
1   2  45.492662   9.192703
2   3  45.430214   9.256324
3   4  45.497771   9.184904
4   5  45.505079   9.093327


In [ ]:
calculate_network_distance(milan_points,milan_metro_stations)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\osmnx\graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(
Calculating shortest distances: 100%|██████████| 10/10 [01:16<00:00,  7.69s/it]


,id,point,latitude,longitude,node,network_distance
0,1,Duomo di Milano,45.4641,9.1919,27653859,394.456
1,2,Sforza Castle,45.4706,9.1796,4660002386,315.642
2,3,Galleria Vittorio Emanuele II,45.4642,9.1900,3179633611,137.506
3,4,Teatro alla Scala,45.4672,9.1896,1829745661,346.871
4,5,Santa Maria delle Grazie,45.4658,9.1702,6425909405,445.029
5,6,Piazza del Duomo,45.4642,9.1900,3179633611,137.506
6,7,Brera Art Gallery,45.4720,9.1885,9031686444,625.574
7,8,San Siro Stadium,45.4781,9.1241,9545171629,203.070
8,9,Porta Nuova,45.4812,9.1919,2737402668,110.461
9,10,Arco della Pace,45.4730,9.1706,6445540335,517.583
